In [6]:
from PIL import Image
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from pathlib import Path


def create_chart(
        csv_filename: str,
        date_column: str,
        chart_type: str,
        columns_to_plot: dict,
        data_folder: str = "data",
        watermark: str = "off",
        title: str = None,
        stacked: str = "off",
        unit: str = None
) -> go.Figure:
    """
    Create a chart from CSV data with specified parameters.
    
    Parameters:
    -----------
    csv_filename : str
        Name of the CSV file (e.g., 'lending_market_rate.csv')
    date_column : str
        Name of the column containing dates to be used as index
    chart_type : str
        Type of chart to create ('line', 'bar', or 'area')
    columns_to_plot : dict
        Dictionary mapping column names to display names 
        (e.g., {'fed_rate': 'Federal Rate', 'defi_rate': 'DeFi Rate'})
    data_folder : str
        Name of the folder containing the CSV files (default: 'data')
    watermark : str
        Whether to add watermark and styling ('on' or 'off', default: 'off')
    title : str
        Title text for the chart (optional)
    stacked : str
        Whether to stack the bars/areas ('on' or 'off', default: 'off')
    unit : str
        Unit for y-axis values ('K', 'M', 'B', or None for no unit formatting)
    """
    # [Previous file loading code remains the same]
    possible_paths = [
        Path.cwd() / data_folder / csv_filename,
        Path.cwd().parent / data_folder / csv_filename,
        Path(data_folder) / csv_filename
    ]

    print(f"Current working directory: {Path.cwd()}")

    df = None
    for path in possible_paths:
        try:
            if path.exists():
                df = pd.read_csv(path)
                print(f"Successfully loaded data from: {path}")
                break
        except Exception as e:
            print(f"Tried path: {path} (not found)")
            continue

    if df is None:
        raise FileNotFoundError(f"Could not find {csv_filename} in any of these locations:\n" +
                                "\n".join(str(p) for p in possible_paths))

    df[date_column] = pd.to_datetime(df[date_column])
    fig = go.Figure()

    def get_hover_template(display_name):
        if unit is None:
            return f"{display_name}: %{{y:,.0f}}<extra></extra>"
        elif unit.upper() == 'K':
            return f"{display_name}: %{{y:,.0f}}K<extra></extra>"
        elif unit.upper() == 'M':
            return f"{display_name}: %{{y:,.1f}}M<extra></extra>"
        elif unit.upper() == 'B':
            return f"{display_name}: %{{y:,.2f}}B<extra></extra>"
        return f"{display_name}: %{{y:,.0f}}<extra></extra>"

    def tick_format(value):
        try:
            val = float(value)
            if unit is None:
                return f"{val:,.0f}"
            elif unit.upper() == 'K':
                return f"{val/1000:,.0f}K"
            elif unit.upper() == 'M':
                return f"{val/1_000_000:,.1f}M"
            elif unit.upper() == 'B':
                return f"{val/1_000_000_000:,.1f}B"
            return f"{val:,.0f}"
        except:
            return value

    # Add traces based on chart type
    for column, display_name in columns_to_plot.items():
        if chart_type.lower() == 'line':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines',
                    hovertemplate=get_hover_template(display_name)
                )
            )
        elif chart_type.lower() == 'bar':
            fig.add_trace(
                go.Bar(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    hovertemplate=get_hover_template(display_name)
                )
            )
        elif chart_type.lower() == 'area':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines',
                    stackgroup='one' if stacked.lower() == 'on' else None,
                    fill='tonexty',
                    hovertemplate=get_hover_template(display_name)
                )
            )
        else:
            raise ValueError("Chart type must be 'line', 'bar', or 'area'")

    # Get min and max values for y-axis
    if stacked.lower() == 'on' and chart_type.lower() in ['bar', 'area']:
        stacked_sums = df[list(columns_to_plot.keys())].sum(axis=1)
        y_min = 0
        y_max = stacked_sums.max()
    else:
        y_min = min(df[column].min() for column in columns_to_plot.keys())
        y_max = max(df[column].max() for column in columns_to_plot.keys())

    # New tick generation logic for billions
    if unit and unit.upper() == 'B':
        # Round down y_min to nearest 10B
        y_min_billions = (y_min / 1_000_000_000)
        y_min_adjusted = np.floor(y_min_billions / 10) * 10 * 1_000_000_000
        
        # Round up y_max to nearest 10B
        y_max_billions = (y_max / 1_000_000_000)
        y_max_adjusted = np.ceil(y_max_billions / 10) * 10 * 1_000_000_000
        
        # Generate ticks at 10B intervals
        tick_vals = np.arange(
            y_min_adjusted,
            y_max_adjusted + 1_000_000_000,  # Add 1B to include the last tick
            10_000_000_000  # 10B intervals
        )
    else:
        # Use original linear spacing for other units
        tick_vals = np.linspace(y_min, y_max, 8)

    tick_texts = [tick_format(val) for val in tick_vals]

    # [Rest of the styling code remains the same]
    axis_size = 17
    border_color = '#7f7f7f'
    default_title = f'{chart_type.capitalize()} Chart: {", ".join(columns_to_plot.values())}'
    if stacked.lower() == 'on' and chart_type.lower() in ['bar', 'area']:
        default_title = f'Stacked {default_title}'

    layout_dict = {
        'title': dict(
            text=title if title else default_title,
            font=dict(
                color='#000000',
                size=29,
                weight='bold'
            ),
            x=0.055,
            y=0.94,
            xanchor='left',
            yanchor='top',
            pad=dict(t=0, b=0)
        ),
        'width': 1350,
        'height': 750,
        'showlegend': True,
        'font': dict(color=border_color),
        'legend': dict(
            orientation="h",
            yanchor="top",
            y=0.98,
            xanchor="left",
            x=0.02,
            font=dict(
                color=border_color,
                size=axis_size
            ),
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        'xaxis': dict(
            title=None,
            tickformat="%b '%y",
            dtick="M3",
            tickangle=0,
            tickmode='array',
            ticktext=[
                date.strftime("%b '%y")
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickvals=[
                date
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            showgrid=False,
            ticks='outside',
            ticklen=8,
            tickwidth=1
        ),
        'yaxis': dict(
            title=None,
            showgrid=False,
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[min(tick_vals), max(tick_vals)],  # Updated to use new tick values
            mirror=True,
            side='left',
            showticksuffix='none',
            showtickprefix='none',
            showticklabels=True,
            tickson='labels',
            tickmode='array',
            ticktext=tick_texts,
            tickvals=tick_vals
        )
    }

    if chart_type.lower() == 'bar' and stacked.lower() == 'on':
        layout_dict['barmode'] = 'stack'

    fig.update_layout(layout_dict)

    # [Watermark code remains the same]
    if watermark.lower() == "on":
        try:
            watermark_img = Image.open("glassnode_large.png")
            fig.add_layout_image(
                dict(
                    source=watermark_img,
                    sizex=0.36,
                    sizey=0.36,
                    xanchor="center",
                    yanchor="middle",
                    xref="paper",
                    yref="paper",
                    x=0.5,
                    y=0.5,
                    opacity=0.15,
                    layer="above"
                )
            )

            fig.add_annotation(
                showarrow=False,
                text=f"© {str(datetime.today().year)} Glassnode. All Rights Reserved",
                font=dict(
                    size=15,
                    color=border_color
                ),
                xref='paper',
                yref='paper',
                x=1,
                y=-0.125,
                opacity=0.5
            )
        except FileNotFoundError:
            print("Warning: Watermark image not found. Skipping watermark but applying styling.")

        fig.update_layout({
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            'xaxis': {
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'showgrid': False,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1
            },
            'yaxis': {
                'showgrid': False,
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1,
                'range': [min(tick_vals), max(tick_vals)],
                'side': 'left',
                'showticksuffix': 'none',
                'showtickprefix': 'none',
                'showticklabels': True,
                'tickson': 'labels'
            },
            'autosize': True
        })

    return fig

In [7]:

# Example usage with unit formatting
fig_area = create_chart(
    csv_filename='lending_market_size_02.csv',
    date_column='t',
    chart_type='line',
    columns_to_plot={
        'total_borrow_7dma': 'Total Borrow (7D-MA) [USD]',
        'total_tvl_7dma': 'Total TVL (7D-MA) [USD]'
    },
    watermark="on",
    title='Lending Market Size Comparison',
    stacked="off",
    unit='B'  # Will format values in millions
)
fig_area.show()

Current working directory: E:\Projects\Glassnode\projects\glassnode_chart\fasanara
Successfully loaded data from: E:\Projects\Glassnode\projects\glassnode_chart\data\lending_market_size_02.csv
